# Wide & Deep

In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('../dataset/ml-100k/data.csv')
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
user = pd.read_csv('../dataset/ml-100k/user.csv')
user.head()

,user_id,age,gender,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
item = pd.read_csv('../dataset/ml-100k/item.csv')
item.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
data
data['like'] = data["rating"].apply(lambda x: x>3.5).astype(int)
data

,user_id,item_id,rating,timestamp,like
0,196,242,3,881250949,0
1,186,302,3,891717742,0
2,22,377,1,878887116,0
3,244,51,2,880606923,0
4,166,346,1,886397596,0
...,...,...,...,...,...
99995,880,476,3,880175444,0
99996,716,204,5,879795543,1
99997,276,1090,1,874795795,0
99998,13,225,2,882399156,0


In [6]:
data

,user_id,item_id,rating,timestamp,like
0,196,242,3,881250949,0
1,186,302,3,891717742,0
2,22,377,1,878887116,0
3,244,51,2,880606923,0
4,166,346,1,886397596,0
...,...,...,...,...,...
99995,880,476,3,880175444,0
99996,716,204,5,879795543,1
99997,276,1090,1,874795795,0
99998,13,225,2,882399156,0


## add features

In [7]:
data = data.merge(user, on='user_id', how='left')
data = data.merge(item, left_on='item_id', right_on='movie_id', how='left')

In [8]:
data

,user_id,item_id,rating,timestamp,like,age,gender,occupation,zipcode,movie_id,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,196,242,3,881250949,0,49,M,writer,55105,242,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,0,39,F,executive,00000,302,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,0,25,M,writer,40206,377,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,0,28,M,technician,80525,51,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,0,47,M,educator,55113,346,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,476,3,880175444,0,13,M,student,83702,476,...,0,0,0,0,0,0,0,0,0,0
99996,716,204,5,879795543,1,36,F,administrator,44265,204,...,0,0,0,0,0,0,1,0,0,0
99997,276,1090,1,874795795,0,21,M,student,95064,1090,...,0,0,0,0,0,0,0,1,0,0
99998,13,225,2,882399156,0,47,M,educator,29206,225,...,0,0,0,0,0,0,0,0,0,0


## data split

In [33]:
def data_split(df, save_to_csv=False):
    # add binary target "like"
    df = make_target(df)
    
    data_sorted = df.sort_values(by=['user_id', 'timestamp'], ascending=[True, False])
    test_set = data_sorted.groupby('user_id').head(5)
    test_indices = test_set.index
    train_set = data_sorted.drop(index=test_indices)
    if save_to_csv:
        train_set.to_csv('../dataset/ml-100k/train_test/train.csv', index=False)
        test_set.to_csv('../dataset/ml-100k/train_test/test.csv', index=False)
    return train_set, test_set


def make_target(df, threshold=3.5):
    df['like'] = df["rating"].apply(lambda x: x>threshold).astype(int)
    df.drop('rating', axis=1, inplace=True)
    
    return df


In [12]:
train_set.shape

(95285, 33)

In [13]:
test_set.shape

(4715, 33)

In [14]:
train_set.to_csv('../dataset/ml-100k/train_test/train.csv', index=False)
test_set.to_csv('../dataset/ml-100k/train_test/test.csv', index=False)

In [38]:
set(test_set.movie_id.unique()) - set(train_set.movie_id.unique())

{1130, 1236, 1525, 1613, 1618, 1624, 1625, 1645, 1650, 1671, 1674}

In [ ]:
there are unseen items in the test set

In [43]:
sorted(train_set.movie_id.unique())[:5]

[1, 2, 3, 4, 5]

THUS, WE ASSIGN 0 TO THE UNSEEN ITEM

In [44]:
training_item_ids = set(train_set.movie_id.unique())

def unknown_item_id_hashing(item_ids, training_item_ids):
    # Check if item_ids is a single item ID (str or int) or a list of item IDs
    if isinstance(item_ids, (str, int)):
        # Return the item ID itself if it exists in training_item_ids, otherwise return 0
        return item_ids if item_ids in training_item_ids else 0
    # If item_ids is a list, apply the check for each item ID in the list
    return [item_id if item_id in training_item_ids else 0 for item_id in item_ids]

In [15]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95285 entries, 88259 to 84323
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             95285 non-null  int64  
 1   item_id             95285 non-null  int64  
 2   rating              95285 non-null  int64  
 3   timestamp           95285 non-null  int64  
 4   like                95285 non-null  int64  
 5   age                 95285 non-null  int64  
 6   gender              95285 non-null  object 
 7   occupation          95285 non-null  object 
 8   zipcode             95285 non-null  object 
 9   movie_id            95285 non-null  int64  
 10  movie_title         95285 non-null  object 
 11  release_date        95276 non-null  object 
 12  video_release_date  0 non-null      float64
 13  IMDb_URL            95272 non-null  object 
 14  unknown             95285 non-null  int64  
 15  Action              95285 non-null  int64  
 16  Adven

## cross product transformation

In [16]:
user.groupby('gender').count().user_id

gender
F    273
M    670
Name: user_id, dtype: int64

In [17]:
user.groupby('occupation').count().user_id.sort_values()

occupation
doctor             7
homemaker          7
none               9
salesman          12
lawyer            12
retired           14
healthcare        16
entertainment     18
marketing         26
technician        27
artist            28
scientist         31
executive         32
writer            45
librarian         51
programmer        66
engineer          67
administrator     79
educator          95
other            105
student          196
Name: user_id, dtype: int64

let's just try to cross-product genre and gender for now, based on intuition

Todo: change to better combination

In [28]:
import numpy as np
def cp_gender_genre(df)：
    genres = ['unknown', 'Action', 'Adventure',
       'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci_Fi', 'Thriller', 'War', 'Western']
    crossed_cols = [['is_male', genre] for genre in genres]
    for a, b in crossed_cols:
        df[a+'-'+b] = np.where((df['gender'] == 'M') & (df[b]==1), 1, 0)
    return df

In [29]:
train_set

,user_id,item_id,rating,timestamp,like,age,gender,occupation,zipcode,movie_id,...,is_male-Fantasy,is_male-Film_Noir,is_male-Horror,is_male-Musical,is_male-Mystery,is_male-Romance,is_male-Sci_Fi,is_male-Thriller,is_male-War,is_male-Western
0,1,111,5,889751711,1,24,M,technician,85711,111,...,0,0,0,0,0,1,0,0,0,0
1,1,242,5,889751633,1,24,M,technician,85711,242,...,0,0,0,0,0,0,0,0,0,0
2,1,189,3,888732928,0,24,M,technician,85711,189,...,0,0,0,0,0,0,0,0,0,0
3,1,32,5,888732909,1,24,M,technician,85711,32,...,0,0,0,0,0,0,0,0,0,0
4,1,209,4,888732908,1,24,M,technician,85711,209,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95280,943,100,5,875501725,1,22,M,student,77841,100,...,0,0,0,0,0,0,0,1,0,0
95281,943,174,4,875410099,1,22,M,student,77841,174,...,0,0,0,0,0,0,0,0,0,0
95282,943,181,4,875409978,1,22,M,student,77841,181,...,0,0,0,0,0,1,1,0,1,0
95283,943,28,4,875409978,1,22,M,student,77841,28,...,0,0,0,0,0,0,0,1,0,0


In [30]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95285 entries, 0 to 95284
Data columns (total 52 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              95285 non-null  int64  
 1   item_id              95285 non-null  int64  
 2   rating               95285 non-null  int64  
 3   timestamp            95285 non-null  int64  
 4   like                 95285 non-null  int64  
 5   age                  95285 non-null  int64  
 6   gender               95285 non-null  object 
 7   occupation           95285 non-null  object 
 8   zipcode              95285 non-null  object 
 9   movie_id             95285 non-null  int64  
 10  movie_title          95285 non-null  object 
 11  release_date         95276 non-null  object 
 12  video_release_date   0 non-null      float64
 13  IMDb_URL             95272 non-null  object 
 14  unknown              95285 non-null  int64  
 15  Action               95285 non-null 

# data loader

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

features = df.drop('target', axis=1).values
target = df['target'].values

features_tensor = torch.tensor(features, dtype=torch.float32)
target_tensor = torch.tensor(target, dtype=torch.float32)  # Use torch.long for classification targets

# Create a TensorDataset
dataset = TensorDataset(features_tensor, target_tensor)


# model

In [31]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.autograd import Variable
from torch.utils.data import DataLoader

class Wide(nn.Module):
    """
    Wide-side consists in simply in "pluging" the features into the output neuron(s)

    Parameters:
    ----------
    wide_dim: int. Number of features per observation
    method  : str. Regression, logistic or multiclass
    n_class : int. number of classes. Defaults to 1 if logistic or regression
    """
    def __init__(self, wide_dim, n_class):

        super(Wide, self).__init__()
        self.wide_dim = wide_dim
        self.n_class = n_class

        self.linear = nn.Linear(self.wide_dim, self.n_class)

    def forward(self,X):

        out = F.sigmoid(self.linear(X))

        return out

In [ ]:
class Deep(nn.Module):
    """
    Deep-side, which consists in a series of embeddings and numerical 
    features passed through a series of dense layers.

    Params:
    --------
    embeddings_input (tuple): 3-elements tuple with the embeddings "set-up" -
    (col_name, unique_values, embeddings dim)
    continuous_cols (list) : list with the name of the continuum columns
    deep_column_idx (dict) : dictionary where the keys are column names and the values
    their corresponding index in the deep-side input tensor
    hidden_layers (list) : list with the number of units per hidden layer
    n_class (int) : number of classes. Defaults to 1 if logistic or regression
    """
    def __init__(self,embeddings_input,continuous_cols,deep_column_idx,hidden_layers,n_class):

        super(Deep, self).__init__()
        self.deep_column_idx = deep_column_idx
        self.embeddings_input = embeddings_input
        self.continuous_cols = continuous_cols
        self.hidden_layers = hidden_layers
        self.n_class = n_class

        # build the embeddings that will be passed through the deep side    
        for col,val,dim in self.embeddings_input:
            setattr(self, 'emb_layer_'+col, nn.Embedding(val, dim))

        # the input dimension to the 1st hidden layer will be the sum of the
        # embeddings dimensions plus the number of continuous features
        input_emb_dim = np.sum([emb[2] for emb in self.embeddings_input])
        self.linear_1 = nn.Linear(input_emb_dim+len(continuous_cols), self.hidden_layers[0])
        for i,h in enumerate(self.hidden_layers[1:],1):
            setattr(self, 'linear_'+str(i+1), nn.Linear( self.hidden_layers[i-1], self.hidden_layers[i] ))

        self.output = nn.Linear(self.hidden_layers[-1], n_class)

    def forward(self, X):

        emb = [getattr(self, 'emb_layer_'+col)(X[:,self.deep_column_idx[col]].long())
               for col,_,_ in self.embeddings_input]

        cont_idx = [self.deep_column_idx[col] for col in self.continuous_cols]
        cont = [X[:, cont_idx].float()]

        deep_inp = torch.cat(emb+cont, 1)

        x_deep = F.relu(self.linear_1(deep_inp))
        for i in range(1,len(self.hidden_layers)):
            x_deep = F.relu( getattr(self, 'linear_'+str(i+1))(x_deep) )

        out = F.sigmoid(self.output(x_deep))

        return out